In [2]:
import pandas as pd
import xarray as xr
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR, OUTPUT_DATA_DIR, RESULTS_DIR

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


In [3]:
# name of output variable
name_deltaT = 'Delta T'

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
percentile = 'percentile'

### Define variables to look at:

In [4]:
# variables to include:

variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    'o3',
    'HFCs',
]
# total ERFs for anthropogenic and total:
variables_erf_tot = ['total_anthropogenic',
                     'total']
variables_all = variables_erf_comp + variables_erf_tot
# Scenarios to plot:
scenarios_fl = ['ssp534-over', 'ssp119', 'ssp460', 'ssp585', 'ssp370',
                'ssp370-lowNTCF-aerchemmip', 'ssp126', 'ssp245', 'ssp434',
                'ssp370-lowNTCF-gidden'
                ]

In [5]:
recommendation = 'recommendation'
IRFpercentiles = [recommendation]
# {'ECS = 2K':0.526, 'ECS = 3.4K':0.884, 'ECS = 5K': 1.136 }

Year to integrate from and to:

In [6]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [7]:
ref_year = 2019

**Years to output change in**

In [8]:
years = [2040, 2100]

### Input dataset:

In [9]:
PATH_DT_INPUT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

In [10]:
PATH_DT_TAB_OUTPUT = RESULTS_DIR / 'tables_recommendation' / 'table_sens_dT_cs_recommandetion.csv'








In [11]:
print(PATH_DT_INPUT)

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_out/dT_data_RCMIP_recommendation.nc


In [12]:
    
    
ds_DT = xr.open_dataset(PATH_DT_INPUT)

In [13]:
ds_DT.variable

<xarray.DataArray 'variable' (variable: 19)>
array(['co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
       'contrails', 'aerosol-radiation_interactions',
       'aerosol-cloud_interactions', 'bc_on_snow', 'land_use', 'volcanic',
       'solar', 'total_anthropogenic', 'total_natural', 'total',
       'aerosol-total', 'aerosol-total-with_bc-snow', 'HFCs'], dtype=object)
Coordinates:
  * variable  (variable) object 'co2' 'ch4' ... 'HFCs'

## Table

### Setup table:

In [14]:

iterables = [list(IRFpercentiles), years]


def setup_table(scenario_n='', variables=variables_all):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    table = pd.DataFrame(columns= variables, index=_i).transpose()
    table.index.name = scenario_n
    return table

In [15]:
# Dicitonary of tables with different ESC:
scntab_dic = {}
for scn in scenarios_fl:
    # Loop over scenrarios
    tab = setup_table(scenario_n=scn, variables=variables_erf_comp)  # make table
    for var in variables_erf_comp:
        # Loop over variables
        tabvar = var.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            # Loop over ESC parameters
            for year in years:
                
                da_ref_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=ref_year,
                ).squeeze()
                
                da_sel_y =ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year
                ) 
                _tab_da = da_sel_y - da_ref_y
                a = float(_tab_da.loc[var].squeeze().values)
                tab.loc[tabvar, (key, year)] = a
    scntab_dic[scn] = tab.copy()

In [16]:
from IPython.display import display

for key in scntab_dic:
    display(scntab_dic[key])

recommendation           
                                         2040       2100
ssp534-over                                             
ch4                                 0.0705118  -0.166599
aerosol-total-with_bc-snow           0.143221   0.384931
aerosol-radiation_interactions      0.0473911  0.0100276
aerosol-cloud_interactions          0.0909345   0.413642
aerosol-total                        0.138326    0.42367
o3                                  0.0446012  -0.133892
HFCs                                 0.044358  0.0168068
bc_on_snow                         0.00489577 -0.0387386
total_anthropogenic                  0.913522   0.947364
total                                0.856145   0.876235

recommendation            
                                         2040        2100
ssp119                                                   
ch4                                 -0.030381   -0.163022
aerosol-total-with_bc-snow             0.2919    0.445276
aerosol-radiation_interactions      0.0352601   0.0444667
aerosol-cloud_interactions           0.284896    0.442839
aerosol-total                        0.320156    0.487306
o3                                 -0.0875336   -0.154183
HFCs                               0.00681278 -0.00254577
bc_on_snow                         -0.0282558  -0.0420298
total_anthropogenic                  0.441451    0.154063
total                                0.384074   0.0829346

recommendation           
                                         2040       2100
ssp460                                                  
ch4                                 0.0929076    0.18682
aerosol-total-with_bc-snow        -0.00502487   0.182846
aerosol-radiation_interactions      0.0238516 -0.0139971
aerosol-cloud_interactions         -0.0476703   0.215471
aerosol-total                      -0.0238187   0.201474
o3                                  0.0586318   0.096969
HFCs                                0.0396849  0.0425885
bc_on_snow                          0.0187938 -0.0186278
total_anthropogenic                  0.688253     2.3306
total                                0.630877    2.25947

recommendation           
                                         2040       2100
ssp585                                                  
ch4                                 0.0663295   0.153352
aerosol-total-with_bc-snow           0.143221   0.254679
aerosol-radiation_interactions      0.0473911  0.0059795
aerosol-cloud_interactions          0.0909345   0.275625
aerosol-total                        0.138326   0.281604
o3                                  0.0419436  0.0526531
HFCs                                0.0443504    0.22836
bc_on_snow                         0.00489577 -0.0269249
total_anthropogenic                    0.8999    4.23102
total                                0.842523     4.1599

recommendation            
                                         2040        2100
ssp370                                                   
ch4                                 0.0816393    0.285313
aerosol-total-with_bc-snow          -0.061394    -0.03541
aerosol-radiation_interactions      0.0183428  0.00531912
aerosol-cloud_interactions          -0.105447  -0.0593361
aerosol-total                      -0.0871037   -0.054017
o3                                  0.0745486    0.243788
HFCs                                0.0281637   0.0998195
bc_on_snow                          0.0257097   0.0186071
total_anthropogenic                  0.700191     3.22891
total                                0.642814     3.15778

recommendation           
                                         2040       2100
ssp370-lowNTCF-aerchemmip                               
ch4                                 0.0816393   0.285313
aerosol-total-with_bc-snow          0.0865772   0.287903
aerosol-radiation_interactions       0.025604  0.0230069
aerosol-cloud_interactions          0.0593218   0.289118
aerosol-total                       0.0849258   0.312125
o3                                  0.0261561   0.161463
HFCs                                0.0281637  0.0998195
bc_on_snow                         0.00165141 -0.0242224
total_anthropogenic                  0.799816    3.45874
total                                 0.74244    3.38761

recommendation             
                                         2040         2100
ssp126                                                    
ch4                                 -0.018502    -0.155442
aerosol-total-with_bc-snow           0.244711     0.454098
aerosol-radiation_interactions      0.0301152    0.0526913
aerosol-cloud_interactions           0.238175     0.441357
aerosol-total                         0.26829     0.494048
o3                                 -0.0643342    -0.153245
HFCs                                 0.018659 -0.000206338
bc_on_snow                         -0.0235787   -0.0399499
total_anthropogenic                  0.549258     0.587645
total                                0.491881     0.516517

recommendation            
                                         2040        2100
ssp245                                                   
ch4                                 0.0461963  0.00250472
aerosol-total-with_bc-snow          0.0567248    0.296192
aerosol-radiation_interactions      0.0111855     0.01173
aerosol-cloud_interactions          0.0458886    0.318932
aerosol-total                       0.0570741    0.330662
o3                                  0.0144792  -0.0476062
HFCs                                0.0303215   0.0599126
bc_on_snow                       -0.000349286  -0.0344692
total_anthropogenic                  0.631925     1.82018
total                                0.574548     1.74905

recommendation            
                                         2040        2100
ssp434                                                   
ch4                                 0.0611096    0.131986
aerosol-total-with_bc-snow          0.0781607    0.266362
aerosol-radiation_interactions      0.0390818  0.00950487
aerosol-cloud_interactions          0.0283535    0.280561
aerosol-total                       0.0674353    0.290066
o3                                  0.0279371   0.0493699
HFCs                               0.00440313 -0.00654308
bc_on_snow                          0.0107253  -0.0237036
total_anthropogenic                  0.571326     1.13808
total                                0.513949     1.06695

recommendation           
                                         2040       2100
ssp370-lowNTCF-gidden                                   
ch4                                -0.0326403  -0.120541
aerosol-total-with_bc-snow          0.0865772   0.287903
aerosol-radiation_interactions       0.025604  0.0230069
aerosol-cloud_interactions          0.0593218   0.289118
aerosol-total                       0.0849258   0.312125
o3                                 -0.0437328 -0.0992323
HFCs                                0.0281684  0.0863879
bc_on_snow                         0.00165141 -0.0242224
total_anthropogenic                  0.606025     2.7216
total                                0.548648    2.65048

### Make table with all scenarios:

In [17]:
iterables = [list(IRFpercentiles), years]
iterables2 = [scenarios_fl, variables_erf_comp]


def setup_table2():  # scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    _r = pd.MultiIndex.from_product(iterables2, names=['', ''])

    table = pd.DataFrame(columns=_r, index=_i).transpose()
    return table

In [18]:
tab = setup_table2()  # scenario_n=scn)

for scn in scenarios_fl:
    for var in variables_erf_comp:
        tabvar = var#.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            for year in years:
                # compute difference between year and ref year
                _da_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year, 
                    variable=var
                ).squeeze()  # .squeeze()
                _da_refy = ds_DT[dtvar].sel(
                    scenario=scn, 
                    year=ref_year, 
                    variable=var,
                ).squeeze()
                # _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                _tab_da = _da_y - _da_refy

                tab.loc[(scn, tabvar), (key, year)] = _tab_da.squeeze().values  # [0]

In [19]:
tab

recommendation  \
                                                                       2040   
                                                                              
ssp534-over           ch4                                0.0705118185891882   
                      aerosol-total-with_bc-snow        0.14322137931713547   
                      aerosol-radiation_interactions   0.047391081758048126   
                      aerosol-cloud_interactions        0.09093452349779296   
                      aerosol-total                      0.1383256052558408   
...                                                                     ...   
ssp370-lowNTCF-gidden o3                              -0.043732827679095226   
                      HFCs                             0.028168381920287824   
                      bc_on_snow                      0.0016514100405829943   
                      total_anthropogenic                0.6060252695770094   
                      total                              0.5486483826907143   

                                                                             
                                                                       2100  
                                                                             
ssp534-over           ch4                               -0.1665994477395357  
                      aerosol-total-with_bc-snow         0.3849311572490604  
                      aerosol-radiation_interactions   0.010027580688094506  
                      aerosol-cloud_interactions         0.4136422156836981  
                      aerosol-total                      0.4236697963717925  
...                                                                     ...  
ssp370-lowNTCF-gidden o3                               -0.09923232596774581  
                      HFCs                                 0.08638792423295  
                      bc_on_snow                      -0.024222441163900557  
                      total_anthropogenic                2.7216041307459724  
                      total                               2.650475324877969  

[100 rows x 2 columns]

In [20]:
tab

recommendation  \
                                                                       2040   
                                                                              
ssp534-over           ch4                                0.0705118185891882   
                      aerosol-total-with_bc-snow        0.14322137931713547   
                      aerosol-radiation_interactions   0.047391081758048126   
                      aerosol-cloud_interactions        0.09093452349779296   
                      aerosol-total                      0.1383256052558408   
...                                                                     ...   
ssp370-lowNTCF-gidden o3                              -0.043732827679095226   
                      HFCs                             0.028168381920287824   
                      bc_on_snow                      0.0016514100405829943   
                      total_anthropogenic                0.6060252695770094   
                      total                              0.5486483826907143   

                                                                             
                                                                       2100  
                                                                             
ssp534-over           ch4                               -0.1665994477395357  
                      aerosol-total-with_bc-snow         0.3849311572490604  
                      aerosol-radiation_interactions   0.010027580688094506  
                      aerosol-cloud_interactions         0.4136422156836981  
                      aerosol-total                      0.4236697963717925  
...                                                                     ...  
ssp370-lowNTCF-gidden o3                               -0.09923232596774581  
                      HFCs                                 0.08638792423295  
                      bc_on_snow                      -0.024222441163900557  
                      total_anthropogenic                2.7216041307459724  
                      total                               2.650475324877969  

[100 rows x 2 columns]

## Save output

In [21]:
tab.to_csv(PATH_DT_TAB_OUTPUT)

In [22]:
PATH_DT_TAB_OUTPUT

PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/results/tables_recommendation/table_sens_dT_cs_recommandetion.csv')